## Explore mitochondrial impairment in tumors 

Annotators used:
- TFLink
- gProfiler
- MitoCart

**Aim**: in this notebook, you will see all the steps for collecting data and constructing a KG to explore mitochondrial impairment in tumer (both in human and mice)

### Import required libraries

In [1]:
# Import modules
import os
import pickle

import pandas as pd

from pyBiodatafuse import id_mapper
from pyBiodatafuse.annotators import gprofiler, mitocarta, tflink
from pyBiodatafuse.graph import generator
from pyBiodatafuse.utils import combine_sources, create_or_append_to_metadata

pd.set_option("display.max_columns", None)

os.makedirs("data", exist_ok=True)
base_dir = os.path.abspath(os.getcwd())

### Load the input files

In [2]:
# Read only specific columns and skip the first row
all_genes = pd.read_excel("datasets/cachexia_vs_control_all_genes.xlsx")
all_genes.rename(columns={"Unnamed: 0": "identifier", "Unnamed: 1": "GENE_SYMBOL"}, inplace=True)
deg_data = all_genes[all_genes["padj"] < 0.01]
print("Number of genes:", len(all_genes["identifier"].unique()))

Number of genes: 12322


In [3]:
all_genes.head()

,identifier,GENE_SYMBOL,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,minus_log10_pvalue
0,ENSG00000000003,TSPAN6,106.899174,-0.401659,0.143909,-2.791057,0.005254,0.077900,2.279542
1,ENSG00000000419,DPM1,601.924666,-0.104654,0.161936,-0.646270,0.518105,0.783377,0.285582
2,ENSG00000000457,SCYL3,244.623536,-0.285418,0.118343,-2.411784,0.015875,0.141951,1.799295
3,ENSG00000000938,FGR,91.958767,0.599304,0.342110,1.751786,0.079811,0.328648,1.097940
4,ENSG00000000971,CFH,499.583125,0.079448,0.255376,0.311101,0.755724,0.907340,0.121637


### Entity resolution with BridgeDB

In [4]:
pickle_path = os.path.join(base_dir, "data/human_bridgedb_df.pkl")
metadata_path = os.path.join(base_dir, "data/human_bridgedb_metadata.pkl")

if not os.path.exists(pickle_path):
    bridgedb_df, bridgedb_metadata = id_mapper.bridgedb_xref(
        identifiers=all_genes,
        input_species="Human",
        input_datasource="Ensembl",
        output_datasource="All",
    )
    bridgedb_df.to_pickle(pickle_path)
    with open(metadata_path, "wb") as file:
        pickle.dump(bridgedb_metadata, file)
else:
    bridgedb_df = pd.read_pickle(pickle_path)
    with open(metadata_path, "rb") as file:
        bridgedb_metadata = pickle.load(file)

In [5]:
print("Number of genes with mapping in BridgeDb:", len(bridgedb_df["identifier"].unique()))
bridgedb_df.head(1)

Number of genes with mapping in BridgeDb: 12310


,identifier,identifier.source,target,target.source,GENE_SYMBOL_dea,baseMean_dea,log2FoldChange_dea,lfcSE_dea,stat_dea,pvalue_dea,padj_dea,minus_log10_pvalue_dea
0,ENSG00000000003,Ensembl,HGNC:11858,HGNC Accession Number,TSPAN6,106.899174,-0.401659,0.143909,-2.791057,0.005254,0.0779,2.279542


### TF-target interactions

Download the "tflink_human.tsv.gz" file found [here](https://cdn.netbiol.org/tflink/download_files/TFLink_Homo_sapiens_interactions_All_simpleFormat_v1.0.tsv.gz).

In [6]:
tflink_path = os.path.join(base_dir, "data/human_tflink.pkl")
tflink_metadata_path = os.path.join(base_dir, "data/human_tflink_metadata.pkl")

if not os.path.exists(tflink_path):
    tflink_df, tflink_metadata = tflink.get_tf_target(
        bridgedb_df=bridgedb_df,
        tf_file=f"{base_dir}/datasets/TFLink_Homo_sapiens_interactions_All_simpleFormat_v1.0.tsv.gz",
        filter_deg=True,
        padj_colname="padj",
        padj_filter=0.05,
    )
    tflink_df.to_pickle(tflink_path)
    with open(tflink_metadata_path, "wb") as file:
        pickle.dump(tflink_metadata, file)
else:
    tflink_df = pd.read_pickle(tflink_path)
    with open(tflink_metadata_path, "rb") as file:
        tflink_metadata = pickle.load(file)

tflink_df.head()

,identifier,identifier.source,target,target.source,GENE_SYMBOL_dea,baseMean_dea,log2FoldChange_dea,lfcSE_dea,stat_dea,pvalue_dea,padj_dea,minus_log10_pvalue_dea,is_tf,is_target,its_target,its_tf
0,ENSG00000000003,Ensembl,7105,NCBI Gene,TSPAN6,106.899174,-0.401659,0.143909,-2.791057,0.005254,0.077900,2.279542,False,True,[],[]
1,ENSG00000000419,Ensembl,8813,NCBI Gene,DPM1,601.924666,-0.104654,0.161936,-0.646270,0.518105,0.783377,0.285582,False,True,[],[]
2,ENSG00000000457,Ensembl,57147,NCBI Gene,SCYL3,244.623536,-0.285418,0.118343,-2.411784,0.015875,0.141951,1.799295,False,True,[],[]
3,ENSG00000000938,Ensembl,2268,NCBI Gene,FGR,91.958767,0.599304,0.342110,1.751786,0.079811,0.328648,1.097940,False,True,[],[]
4,ENSG00000000971,Ensembl,3075,NCBI Gene,CFH,499.583125,0.079448,0.255376,0.311101,0.755724,0.907340,0.121637,False,True,[],[]


### Enrichment analysis using g:Profiler
all the pathways and annotations are being added despite being significance.

In [7]:
gprofiler_path = os.path.join(base_dir, "data/human_gprofiler.pkl")
gprofiler_metadata_path = os.path.join(base_dir, "data/human_gprofiler_metadata.pkl")

if not os.path.exists(gprofiler_path):
    gprofiler_df, gprofiler_metadata = gprofiler.get_gene_enrichment(
        bridgedb_df=bridgedb_df, species="hsapiens", padj_colname="padj", padj_filter=0.05
    )
    gprofiler_df.to_pickle(gprofiler_path)
    with open(gprofiler_metadata_path, "wb") as file:
        pickle.dump(gprofiler_metadata, file)
else:
    gprofiler_df = pd.read_pickle(gprofiler_path)
    with open(gprofiler_metadata_path, "rb") as file:
        gprofiler_metadata = pickle.load(file)

gprofiler_df.head(1)

,identifier,identifier.source,target,target.source,GENE_SYMBOL_dea,baseMean_dea,log2FoldChange_dea,lfcSE_dea,stat_dea,pvalue_dea,padj_dea,minus_log10_pvalue_dea,g:Profiler_corum,g:Profiler_go:bp,g:Profiler_go:cc,g:Profiler_go:mf,g:Profiler_hp,g:Profiler_hpa,g:Profiler_kegg,g:Profiler_mirna,g:Profiler_reac,g:Profiler_tf,g:Profiler_wp
0,ENSG00000000003,Ensembl,ENSG00000000003,Ensembl,TSPAN6,106.899174,-0.401659,0.143909,-2.791057,0.005254,0.0779,2.279542,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
gprofiler_df[gprofiler_df["g:Profiler_reac"].notna()].head(1)

,identifier,identifier.source,target,target.source,GENE_SYMBOL_dea,baseMean_dea,log2FoldChange_dea,lfcSE_dea,stat_dea,pvalue_dea,padj_dea,minus_log10_pvalue_dea,g:Profiler_corum,g:Profiler_go:bp,g:Profiler_go:cc,g:Profiler_go:mf,g:Profiler_hp,g:Profiler_hpa,g:Profiler_kegg,g:Profiler_mirna,g:Profiler_reac,g:Profiler_tf,g:Profiler_wp
6,ENSG00000001084,Ensembl,ENSG00000001084,Ensembl,GCLC,412.449345,0.454035,0.1302,3.487222,0.000488,0.018659,3.311521,"[{'id': 'CORUM:7186', 'name': 'Glutamate-cyste...","[{'id': 'GO:0044281', 'name': 'small molecule ...","[{'id': 'GO:0005739', 'name': 'mitochondrion',...","[{'id': 'GO:0003824', 'name': 'catalytic activ...","[{'id': 'HP:0012337', 'name': 'Abnormal homeos...","[{'id': 'HPA:0440000', 'name': 'skeletal muscl...","[{'id': 'KEGG:01100', 'name': 'Metabolic pathw...","[{'id': 'MIRNA:hsa-miR-30e-5p', 'name': 'hsa-m...","[{'id': 'REAC:R-HSA-1430728', 'name': 'Metabol...","[{'id': 'TF:M07428', 'name': 'Factor: Six-3; m...","[{'id': 'WP:WP2882', 'name': 'Nuclear receptor..."


### Add MitoCarta data

In [9]:
mitocarta_path = os.path.join(base_dir, "data/human_mitocarta.pkl")
mitocarta_metadata_path = os.path.join(base_dir, "data/human_mitocarta_metadata.pkl")

if not os.path.exists(mitocarta_path):
    mitocarta_df, mitocarta_metadata = mitocarta.get_gene_mito_pathways(
        bridgedb_df=bridgedb_df,
        mitocarta_file="Human.MitoCarta3.0.xls",
        filename="data/human_mitocarta3.0_human.xls",
        species="hsapiens",
        sheet_name="A Human MitoCarta3.0",
    )
    mitocarta_df.to_pickle(mitocarta_path)
    with open(mitocarta_metadata_path, "wb") as file:
        pickle.dump(mitocarta_metadata, file)
else:
    mitocarta_df = pd.read_pickle(mitocarta_path)
    with open(mitocarta_metadata_path, "rb") as file:
        mitocarta_metadata = pickle.load(file)

mitocarta_df.head()

,identifier,identifier.source,target,target.source,GENE_SYMBOL_dea,baseMean_dea,log2FoldChange_dea,lfcSE_dea,stat_dea,pvalue_dea,padj_dea,minus_log10_pvalue_dea,MitoCarta_pathways
0,ENSG00000000003,Ensembl,ENSG00000000003,Ensembl,TSPAN6,106.899174,-0.401659,0.143909,-2.791057,0.005254,0.077900,2.279542,"[{'gene_description': nan, 'evidence': nan, 's..."
1,ENSG00000000419,Ensembl,ENSG00000000419,Ensembl,DPM1,601.924666,-0.104654,0.161936,-0.646270,0.518105,0.783377,0.285582,"[{'gene_description': nan, 'evidence': nan, 's..."
2,ENSG00000000457,Ensembl,ENSG00000000457,Ensembl,SCYL3,244.623536,-0.285418,0.118343,-2.411784,0.015875,0.141951,1.799295,"[{'gene_description': nan, 'evidence': nan, 's..."
3,ENSG00000000938,Ensembl,ENSG00000000938,Ensembl,FGR,91.958767,0.599304,0.342110,1.751786,0.079811,0.328648,1.097940,"[{'gene_description': nan, 'evidence': nan, 's..."
4,ENSG00000000971,Ensembl,ENSG00000000971,Ensembl,CFH,499.583125,0.079448,0.255376,0.311101,0.755724,0.907340,0.121637,"[{'gene_description': nan, 'evidence': nan, 's..."


In [10]:
mitocarta_df[mitocarta_df["identifier"] == "ENSG00000005156"]["MitoCarta_pathways"].to_dict()

{65: [{'gene_description': 'DNA ligase 3',
   'evidence': 'literature, targetP signal, mito protein domain, coexpression',
   'sub_mito_localization': 'Matrix',
   'hpa_location': 'Nucleoplasm (Supported)',
   'tissue_expression': nan,
   'mito_pathways': 'mtDNA repair'}]}

## Graph generation 

### Combine all data and metadata

In [11]:
combined_df = combine_sources(
    bridgedb_df,
    [
        tflink_df,
        mitocarta_df,
        gprofiler_df,
    ],
)
combined_df.head()

,identifier,identifier.source,target,target.source,GENE_SYMBOL_dea,baseMean_dea,log2FoldChange_dea,lfcSE_dea,stat_dea,pvalue_dea,padj_dea,minus_log10_pvalue_dea,is_tf,is_target,its_target,its_tf,MitoCarta_pathways,g:Profiler_corum,g:Profiler_go:bp,g:Profiler_go:cc,g:Profiler_go:mf,g:Profiler_hp,g:Profiler_hpa,g:Profiler_kegg,g:Profiler_mirna,g:Profiler_reac,g:Profiler_tf,g:Profiler_wp
0,ENSG00000000003,Ensembl,ENSG00000000003,Ensembl,TSPAN6,106.899174,-0.401659,0.143909,-2.791057,0.005254,0.077900,2.279542,False,True,[],[],"[{'gene_description': nan, 'evidence': nan, 's...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ENSG00000000419,Ensembl,ENSG00000000419,Ensembl,DPM1,601.924666,-0.104654,0.161936,-0.646270,0.518105,0.783377,0.285582,False,True,[],[],"[{'gene_description': nan, 'evidence': nan, 's...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ENSG00000000457,Ensembl,ENSG00000000457,Ensembl,SCYL3,244.623536,-0.285418,0.118343,-2.411784,0.015875,0.141951,1.799295,False,True,[],[],"[{'gene_description': nan, 'evidence': nan, 's...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ENSG00000000938,Ensembl,ENSG00000000938,Ensembl,FGR,91.958767,0.599304,0.342110,1.751786,0.079811,0.328648,1.097940,False,True,[],[],"[{'gene_description': nan, 'evidence': nan, 's...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ENSG00000000971,Ensembl,ENSG00000000971,Ensembl,CFH,499.583125,0.079448,0.255376,0.311101,0.755724,0.907340,0.121637,False,True,[],[],"[{'gene_description': nan, 'evidence': nan, 's...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
combined_df_path = os.path.join(base_dir, "data/human_combined_df.pkl")

if not os.path.exists(combined_df_path):
    combined_df.to_pickle(combined_df_path)
else:
    with open(combined_df_path, "rb") as f:
        combined_df = pickle.load(f)

combined_df.head()

,identifier,identifier.source,target,target.source,GENE_SYMBOL_dea,baseMean_dea,log2FoldChange_dea,lfcSE_dea,stat_dea,pvalue_dea,padj_dea,minus_log10_pvalue_dea,is_tf,is_target,its_target,its_tf,MitoCarta_pathways,g:Profiler_corum,g:Profiler_go:bp,g:Profiler_go:cc,g:Profiler_go:mf,g:Profiler_hp,g:Profiler_hpa,g:Profiler_kegg,g:Profiler_mirna,g:Profiler_reac,g:Profiler_tf,g:Profiler_wp
0,ENSG00000000003,Ensembl,ENSG00000000003,Ensembl,TSPAN6,106.899174,-0.401659,0.143909,-2.791057,0.005254,0.077900,2.279542,False,True,[],[],"[{'gene_description': nan, 'evidence': nan, 's...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ENSG00000000419,Ensembl,ENSG00000000419,Ensembl,DPM1,601.924666,-0.104654,0.161936,-0.646270,0.518105,0.783377,0.285582,False,True,[],[],"[{'gene_description': nan, 'evidence': nan, 's...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ENSG00000000457,Ensembl,ENSG00000000457,Ensembl,SCYL3,244.623536,-0.285418,0.118343,-2.411784,0.015875,0.141951,1.799295,False,True,[],[],"[{'gene_description': nan, 'evidence': nan, 's...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ENSG00000000938,Ensembl,ENSG00000000938,Ensembl,FGR,91.958767,0.599304,0.342110,1.751786,0.079811,0.328648,1.097940,False,True,[],[],"[{'gene_description': nan, 'evidence': nan, 's...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ENSG00000000971,Ensembl,ENSG00000000971,Ensembl,CFH,499.583125,0.079448,0.255376,0.311101,0.755724,0.907340,0.121637,False,True,[],[],"[{'gene_description': nan, 'evidence': nan, 's...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
combined_metadata = create_or_append_to_metadata(
    bridgedb_metadata,
    [
        tflink_metadata,
        mitocarta_metadata,
        gprofiler_metadata,
    ],
)

In [14]:
with open("data/human_combined_metadata.pkl", "wb") as out:
    pickle.dump(combined_metadata, out)

### Create a graph from the annotated dataframe

#### subseting the rows to contruct the graph based on the biological quetion

In [15]:
# Extract all 'NCBI.GeneID.TF' values into a single list
ncbi_gene_ids = (
    combined_df["its_tf"]
    .apply(lambda x: [d["NCBI.GeneID.TF"] for d in x] if isinstance(x, list) else [])
    .explode()
    .dropna()
    .unique()
    .tolist()
)

len(ncbi_gene_ids)

997

In [16]:
combined_df_tf = combined_df[combined_df["target"].isin(ncbi_gene_ids)]
combined_df_sig = combined_df[combined_df["padj_dea"] <= 0.05]
combined_df_sig = combined_df_sig[~combined_df_sig["target"].isin(ncbi_gene_ids)]
combined_df_sig.shape

(594, 28)

In [17]:
combined_df_tf_sig = pd.concat([combined_df_sig, combined_df_tf], axis=0, ignore_index=True)
combined_df_tf_sig.shape

(594, 28)

In [18]:
combined_df[combined_df["identifier"] == "ENSG00000108387"]["MitoCarta_pathways"]

2565    [{'gene_description': 'septin 4', 'evidence': ...
Name: MitoCarta_pathways, dtype: object

In [19]:
pygraph = generator.build_networkx_graph(combined_df)

Building graph: 100%|██████████| 12404/12404 [00:04<00:00, 2928.06it/s]


In [20]:
from collections import defaultdict

sources = defaultdict(int)
edge_types = defaultdict(int)
for snode, dnode, data in pygraph.edges(data=True):
    sources[data["datasource"]] += 1
    edge_types[data["label"]] += 1

sources

defaultdict(int, {'g:Profiler': 738648, 'MitoCarta': 906})

In [21]:
edge_types

defaultdict(int,
            {'part_of': 65623,
             'linked_to': 36273,
             'expressed_in': 57636,
             'regulated_by': 566654,
             'regulates': 12462,
             'encodes_mitochondrial_protein': 906})